In [1]:
print("hello")

hello


In [2]:
%pwd

'e:\\Medical-Chatbot-Gen-AI\\research'

In [3]:
import os

os.chdir("../")

In [4]:
%pwd

'e:\\Medical-Chatbot-Gen-AI'

In [7]:
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter  


In [8]:
def load_pdf_file(data):
    loader = DirectoryLoader(data, glob="*.pdf", loader_cls=PyPDFLoader)
    documents = loader.load()   
    return documents

In [11]:
extracted_doc = load_pdf_file("Data/")

In [ ]:
# extracted_doc

In [40]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

def text_split(extracted_doc):
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=500,
        chunk_overlap=20,
        # separators=[
        #     "\nDefinition\n",
        #     "\nDescription\n",
        #     "\nCauses and symptoms\n",
        #     "\nDiagnosis\n",
        #     "\nTreatment\n",
        #     "\nPrognosis\n",
        #     "\n\n",
        #     "\n",
        #     " "
        # ]
    )
    text_chunks = text_splitter.split_documents(extracted_doc)
    return text_chunks


In [41]:
text_chunks = text_split(extracted_doc)
print("Length of text chunks:", len(text_chunks))

Length of text chunks: 6983


In [37]:
text_chunks 

[Document(page_content='TheGALE\nENCYCLOPEDIA\nofMEDICINE\nSECOND EDITION', metadata={'source': 'Data\\resource.pdf', 'page': 0}),
 Document(page_content='TheGALE\nENCYCLOPEDIA\nofMEDICINE\nSECOND EDITION\nJACQUELINE L. LONGE, EDITOR\nDEIRDRE S. BLANCHFIELD, ASSOCIATE EDITOR\nVOLUME\nC-F2', metadata={'source': 'Data\\resource.pdf', 'page': 1}),
 Document(page_content='STAFF\nJacqueline L. Longe, Project Editor\nDeirdre S. Blanchfield, Associate Editor\nChristine B. Jeryan, Managing Editor\nDonna Olendorf, Senior Editor\nStacey Blachford, Associate Editor\nKate Kretschmann, Melissa C. McDade, Ryan\nThomason, Assistant Editors\nMark Springer, Technical Specialist\nAndrea Lopeman, Programmer/Analyst\nBarbara J. Yarrow, Manager, Imaging and Multimedia\nContent\nRobyn V . Young, Project Manager, Imaging and\nMultimedia Content\nDean Dauphinais, Senior Editor, Imaging and\nMultimedia Content\nKelly A. Quin, Editor, Imaging and Multimedia Content', metadata={'source': 'Data\\resource.pdf', 'p

In [49]:
from langchain.embeddings import HuggingFaceEmbeddings

In [50]:
def download_hugging_face_embeddings():
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    return embeddings

In [51]:
embeddings = download_hugging_face_embeddings()

In [53]:
query_result = embeddings.embed_query("hello")
print("Query Result:", query_result)

Query Result: [-0.0627717673778534, 0.05495880916714668, 0.052164819091558456, 0.0857899934053421, -0.08274887502193451, -0.07457298785448074, 0.06855471432209015, 0.01839643530547619, -0.08201131224632263, -0.03738483041524887, 0.012124921195209026, 0.0035183315631002188, -0.004134307615458965, -0.04378442093729973, 0.021807294338941574, -0.005102717317640781, 0.01954657956957817, -0.042348749935626984, -0.11035961657762527, 0.005424536298960447, -0.05573476850986481, 0.028052471578121185, -0.02315874584019184, 0.028481338173151016, -0.05370965227484703, -0.05260162428021431, 0.03393922746181488, 0.04538865387439728, 0.02371840737760067, -0.07312081009149551, 0.05477771535515785, 0.017047284170985222, 0.08136036247015, -0.0028627102728933096, 0.011958105489611626, 0.07355856150388718, -0.09423744678497314, -0.0813620314002037, 0.040015414357185364, 0.0006921715685166419, -0.013393287546932697, -0.054538071155548096, 0.005151416640728712, -0.026139788329601288, 0.036807116121053696, -0